# 📊 Análisis Exploratorio de Proyectos de Ley - Congreso del Perú

Este notebook contiene el análisis exploratorio de los proyectos de ley extraídos del Congreso del Perú.

## 📋 Contenido
1. **Carga y Limpieza de Datos**
2. **Análisis Descriptivo**
3. **Análisis Temporal**
4. **Análisis por Partido Político**
5. **Análisis por Tipo de Proyecto**
6. **Análisis de Autores**
7. **Visualizaciones Interactivas**


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import glob
import os
import sys
import json
from pathlib import Path

# Agregar el directorio padre al path para importar utils
sys.path.append('..')
from utils.limpieza import DataCleaner
from utils.data_validator import DataValidator
from utils.performance_monitor import PerformanceMonitor

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurar pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Configurar plotly para mostrar gráficos inline
import plotly.io as pio
pio.renderers.default = "notebook"

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de análisis: {datetime.now().strftime('%d/%m/%Y %H:%M')}")
print(f"🐍 Python version: {sys.version}")

# Inicializar componentes
cleaner = DataCleaner()
validator = DataValidator()
print("🔧 Componentes inicializados")


## 1. 📂 Carga y Limpieza de Datos


In [ ]:
# Cargar todos los archivos CSV disponibles
data_files = glob.glob('../data/proyectos_ley_*.csv')
print(f"📁 Archivos encontrados: {len(data_files)}")
for file in data_files:
    print(f"   - {file}")

# Cargar y combinar todos los datos
dataframes = []
for file in data_files:
    try:
        df = pd.read_csv(file, encoding='utf-8-sig')
        df['archivo_origen'] = os.path.basename(file)
        dataframes.append(df)
        print(f"✅ Cargado: {file} ({len(df)} registros)")
    except Exception as e:
        print(f"❌ Error cargando {file}: {e}")

# Combinar todos los dataframes
if dataframes:
    df_raw = pd.concat(dataframes, ignore_index=True)
    print(f"\n📊 Total de registros combinados: {len(df_raw)}")
    print(f"📅 Columnas disponibles: {list(df_raw.columns)}")
else:
    print("❌ No se encontraron archivos de datos")
    df_raw = pd.DataFrame()


In [ ]:
# Aplicar limpieza de datos
if not df_raw.empty:
    print("🧹 Aplicando limpieza de datos...")
    cleaner = DataCleaner()
    df_clean = cleaner.limpiar_dataframe(df_raw)
    
    print(f"✅ Limpieza completada")
    print(f"📊 Registros después de limpieza: {len(df_clean)}")
    
    # Mostrar primeras filas
    print("\n🔍 Primeras 5 filas de datos limpios:")
    display(df_clean.head())
    
    # Mostrar información básica
    print(f"\n📋 Información del dataset:")
    print(f"   - Forma: {df_clean.shape}")
    print(f"   - Memoria utilizada: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   - Valores nulos por columna:")
    null_counts = df_clean.isnull().sum()
    for col, count in null_counts[null_counts > 0].items():
        print(f"     * {col}: {count}")
else:
    print("❌ No hay datos para limpiar")
    df_clean = pd.DataFrame()


## 2. 🔍 Validación y Calidad de Datos


In [ ]:
# Validar calidad de datos
if not df_raw.empty:
    print("🔍 Iniciando validación de calidad de datos...")
    
    # Validar datos raw
    df_validated, validation_report = validator.validate_dataframe(df_raw)
    
    print(f"\n📊 REPORTE DE VALIDACIÓN:")
    print(f"   Total registros: {validation_report['total_records']}")
    print(f"   Registros válidos: {validation_report['valid_records']}")
    print(f"   Registros inválidos: {validation_report['invalid_records']}")
    print(f"   Total errores: {validation_report['total_errors']}")
    print(f"   Total advertencias: {validation_report['total_warnings']}")
    
    if validation_report['field_errors']:
        print(f"\n🔴 Errores por campo:")
        for field, count in validation_report['field_errors'].items():
            print(f"   {field}: {count}")
    
    if validation_report['field_warnings']:
        print(f"\n🟡 Advertencias por campo:")
        for field, count in validation_report['field_warnings'].items():
            print(f"   {field}: {count}")
    
    # Guardar reporte de validación
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    validation_file = f'../reports/validation_report_{timestamp}.json'
    Path('../reports').mkdir(exist_ok=True)
    
    with open(validation_file, 'w', encoding='utf-8') as f:
        json.dump(validation_report, f, indent=2, default=str)
    
    print(f"\n💾 Reporte de validación guardado en: {validation_file}")
else:
    print("❌ No hay datos para validar")


## 3. 📤 Exportar datos limpios y resultados


In [ ]:
# Función para exportar datos y generar reportes
def export_analysis_results(df_clean, resumen, timestamp):
    """Exporta datos limpios y genera reportes automáticos"""
    
    # Crear directorios necesarios
    Path('../analysis').mkdir(exist_ok=True)
    Path('../visualizations').mkdir(exist_ok=True)
    Path('../reports').mkdir(exist_ok=True)
    
    # 1. Exportar datos limpios
    clean_file = f'../data/proyectos_ley_limpios_{timestamp}.csv'
    df_clean.to_csv(clean_file, index=False, encoding='utf-8-sig')
    print(f"✅ Datos limpios exportados: {clean_file}")
    
    # 2. Exportar resumen como JSON
    summary_file = f'../analysis/resumen_analisis_{timestamp}.json'
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(resumen, f, indent=2, default=str)
    print(f"✅ Resumen exportado: {summary_file}")
    
    # 3. Generar reporte HTML
    html_file = f'../reports/reporte_analisis_{timestamp}.html'
    generate_html_report(resumen, html_file, timestamp)
    print(f"✅ Reporte HTML generado: {html_file}")
    
    # 4. Exportar estadísticas por partido
    if 'partido_politico' in df_clean.columns:
        party_stats = df_clean['partido_politico'].value_counts().reset_index()
        party_stats.columns = ['partido', 'proyectos']
        party_file = f'../analysis/estadisticas_partidos_{timestamp}.csv'
        party_stats.to_csv(party_file, index=False, encoding='utf-8-sig')
        print(f"✅ Estadísticas por partido: {party_file}")
    
    # 5. Exportar estadísticas por tipo de proyecto
    if 'tipo_proyecto' in df_clean.columns:
        type_stats = df_clean['tipo_proyecto'].value_counts().reset_index()
        type_stats.columns = ['tipo', 'proyectos']
        type_file = f'../analysis/estadisticas_tipos_{timestamp}.csv'
        type_stats.to_csv(type_file, index=False, encoding='utf-8-sig')
        print(f"✅ Estadísticas por tipo: {type_file}")
    
    return {
        'clean_data': clean_file,
        'summary': summary_file,
        'html_report': html_file,
        'party_stats': party_file if 'partido_politico' in df_clean.columns else None,
        'type_stats': type_file if 'tipo_proyecto' in df_clean.columns else None
    }

def generate_html_report(resumen, output_file, timestamp):
    """Genera un reporte HTML interactivo"""
    
    html_content = f"""
    <!DOCTYPE html>
    <html lang="es">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Reporte de Análisis - Proyectos de Ley del Perú</title>
        <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f5f5f5;
                color: #333;
            }}
            .container {{
                max-width: 1200px;
                margin: 0 auto;
                background: white;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            }}
            .header {{
                text-align: center;
                border-bottom: 3px solid #2196F3;
                padding-bottom: 20px;
                margin-bottom: 30px;
            }}
            .header h1 {{
                color: #2196F3;
                margin: 0;
                font-size: 2.5em;
            }}
            .header p {{
                color: #666;
                margin: 10px 0 0 0;
                font-size: 1.1em;
            }}
            .stats-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                gap: 20px;
                margin: 30px 0;
            }}
            .stat-card {{
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 25px;
                border-radius: 10px;
                text-align: center;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }}
            .stat-value {{
                font-size: 2.5em;
                font-weight: bold;
                margin-bottom: 10px;
            }}
            .stat-label {{
                font-size: 1.1em;
                opacity: 0.9;
            }}
            .section {{
                margin: 40px 0;
                padding: 25px;
                background: #f8f9fa;
                border-radius: 8px;
                border-left: 5px solid #2196F3;
            }}
            .section h2 {{
                color: #2196F3;
                margin-top: 0;
                font-size: 1.8em;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 20px 0;
                background: white;
                border-radius: 8px;
                overflow: hidden;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }}
            th, td {{
                padding: 12px 15px;
                text-align: left;
                border-bottom: 1px solid #ddd;
            }}
            th {{
                background: #2196F3;
                color: white;
                font-weight: 600;
            }}
            tr:hover {{
                background-color: #f5f5f5;
            }}
            .footer {{
                text-align: center;
                margin-top: 40px;
                padding-top: 20px;
                border-top: 1px solid #ddd;
                color: #666;
            }}
            .badge {{
                display: inline-block;
                padding: 4px 8px;
                background: #4CAF50;
                color: white;
                border-radius: 4px;
                font-size: 0.8em;
                margin-left: 10px;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>🇵🇪 Reporte de Análisis</h1>
                <p>Proyectos de Ley del Congreso del Perú</p>
                <p>Generado el: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}</p>
            </div>
            
            <div class="stats-grid">
                <div class="stat-card">
                    <div class="stat-value">{resumen.get('total_projects', 0)}</div>
                    <div class="stat-label">Total Proyectos</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{resumen.get('promedio_autores', 0):.1f}</div>
                    <div class="stat-label">Promedio Autores</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{len(resumen.get('proyectos_por_partido', {}))}</div>
                    <div class="stat-label">Partidos Políticos</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{len(resumen.get('proyectos_por_tipo', {}))}</div>
                    <div class="stat-label">Tipos de Proyecto</div>
                </div>
            </div>
            
            <div class="section">
                <h2>📊 Resumen Ejecutivo</h2>
                <p><strong>Período de análisis:</strong> {resumen.get('fecha_inicio', 'N/A')} - {resumen.get('fecha_fin', 'N/A')}</p>
                <p><strong>Congresista más activo:</strong> {resumen.get('congresista_mas_activo', 'N/A')}</p>
            </div>
            
            <div class="section">
                <h2>🏛️ Proyectos por Partido Político</h2>
                <table>
                    <thead>
                        <tr>
                            <th>Partido</th>
                            <th>Proyectos</th>
                            <th>Porcentaje</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    # Agregar datos de partidos
    total_projects = resumen.get('total_projects', 1)
    for partido, count in list(resumen.get('proyectos_por_partido', {}).items())[:10]:
        percentage = (count / total_projects) * 100
        html_content += f"""
                        <tr>
                            <td>{partido}</td>
                            <td>{count}</td>
                            <td>{percentage:.1f}%</td>
                        </tr>
        """
    
    html_content += """
                    </tbody>
                </table>
            </div>
            
            <div class="section">
                <h2>📋 Proyectos por Tipo</h2>
                <table>
                    <thead>
                        <tr>
                            <th>Tipo</th>
                            <th>Proyectos</th>
                            <th>Porcentaje</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    # Agregar datos de tipos
    for tipo, count in list(resumen.get('proyectos_por_tipo', {}).items())[:10]:
        percentage = (count / total_projects) * 100
        html_content += f"""
                        <tr>
                            <td>{tipo}</td>
                            <td>{count}</td>
                            <td>{percentage:.1f}%</td>
                        </tr>
        """
    
    html_content += f"""
                    </tbody>
                </table>
            </div>
            
            <div class="footer">
                <p>Reporte generado automáticamente por Peru Congress Laws Scraper v2.0</p>
                <p>Timestamp: {timestamp}</p>
            </div>
        </div>
    </body>
    </html>
    """
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)

# Ejecutar exportación si hay datos limpios
if 'df_clean' in locals() and not df_clean.empty:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    if 'resumen' in locals():
        exported_files = export_analysis_results(df_clean, resumen, timestamp)
        print(f"\n🎉 Exportación completada exitosamente!")
        print(f"📁 Archivos generados:")
        for key, file_path in exported_files.items():
            if file_path:
                print(f"   - {key}: {file_path}")
    else:
        print("⚠️ No hay resumen disponible para exportar")
else:
    print("⚠️ No hay datos limpios para exportar")


## 2. 📊 Análisis Descriptivo


In [ ]:
# Generar resumen estadístico
if not df_clean.empty:
    resumen = cleaner.generar_resumen(df_clean)
    
    print("📊 RESUMEN ESTADÍSTICO")
    print("=" * 50)
    print(f"📈 Total de proyectos: {resumen['total_proyectos']}")
    print(f"📅 Período: {resumen['fecha_inicio'].strftime('%d/%m/%Y')} - {resumen['fecha_fin'].strftime('%d/%m/%Y')}")
    print(f"👥 Promedio de autores por proyecto: {resumen['promedio_autores']:.1f}")
    print(f"🏆 Congresista más activo: {resumen['congresista_mas_activo']}")
    
    print(f"\n🏛️ PROYECTOS POR PARTIDO POLÍTICO:")
    for partido, count in list(resumen['proyectos_por_partido'].items())[:10]:
        print(f"   {partido}: {count}")
    
    print(f"\n📋 PROYECTOS POR TIPO:")
    for tipo, count in list(resumen['proyectos_por_tipo'].items())[:10]:
        print(f"   {tipo}: {count}")
    
    print(f"\n🌍 PROYECTOS POR REGIÓN:")
    for region, count in list(resumen['proyectos_por_region'].items())[:10]:
        print(f"   {region}: {count}")
else:
    print("❌ No hay datos para análisis")


## 3. 📈 Visualizaciones Interactivas


In [ ]:
# Crear visualizaciones interactivas
if not df_clean.empty:
    
    # 1. Proyectos por partido político
    fig1 = px.bar(
        x=list(resumen['proyectos_por_partido'].keys())[:10],
        y=list(resumen['proyectos_por_partido'].values())[:10],
        title="📊 Proyectos de Ley por Partido Político (Top 10)",
        labels={'x': 'Partido Político', 'y': 'Número de Proyectos'},
        color=list(resumen['proyectos_por_partido'].values())[:10],
        color_continuous_scale='viridis'
    )
    fig1.update_layout(
        xaxis_tickangle=-45,
        height=500,
        showlegend=False
    )
    fig1.show()
    
    # 2. Proyectos por tipo
    fig2 = px.pie(
        values=list(resumen['proyectos_por_tipo'].values()),
        names=list(resumen['proyectos_por_tipo'].keys()),
        title="📋 Distribución de Proyectos por Tipo"
    )
    fig2.update_traces(textposition='inside', textinfo='percent+label')
    fig2.show()
    
    # 3. Proyectos por región
    if resumen['proyectos_por_region']:
        fig3 = px.bar(
            x=list(resumen['proyectos_por_region'].keys())[:10],
            y=list(resumen['proyectos_por_region'].values())[:10],
            title="🌍 Proyectos de Ley por Región (Top 10)",
            labels={'x': 'Región', 'y': 'Número de Proyectos'},
            color=list(resumen['proyectos_por_region'].values())[:10],
            color_continuous_scale='plasma'
        )
        fig3.update_layout(
            xaxis_tickangle=-45,
            height=500,
            showlegend=False
        )
        fig3.show()
    
    # 4. Timeline de proyectos
    if 'fecha_datetime' in df_clean.columns:
        df_timeline = df_clean.groupby(df_clean['fecha_datetime'].dt.date).size().reset_index()
        df_timeline.columns = ['fecha', 'proyectos']
        
        fig4 = px.line(
            df_timeline,
            x='fecha',
            y='proyectos',
            title="📅 Timeline de Proyectos de Ley",
            labels={'fecha': 'Fecha', 'proyectos': 'Número de Proyectos'}
        )
        fig4.update_layout(height=400)
        fig4.show()
    
    print("✅ Visualizaciones generadas correctamente")
else:
    print("❌ No hay datos para visualizar")


## 4. 🔍 Análisis Detallado


In [ ]:
# Análisis detallado de autores y colaboraciones
if not df_clean.empty:
    
    # Análisis de autores más activos
    print("👥 ANÁLISIS DE AUTORES")
    print("=" * 40)
    
    # Contar apariciones de cada autor
    todos_autores = []
    for autores_list in df_clean['autores_limpios']:
        todos_autores.extend(autores_list)
    
    if todos_autores:
        conteo_autores = pd.Series(todos_autores).value_counts()
        
        print(f"📊 Top 10 congresistas más activos:")
        for i, (autor, count) in enumerate(conteo_autores.head(10).items(), 1):
            print(f"   {i:2d}. {autor}: {count} proyectos")
        
        # Análisis de colaboraciones
        print(f"\n🤝 ANÁLISIS DE COLABORACIONES")
        print("=" * 40)
        
        # Contar proyectos con múltiples autores
        proyectos_multiples = df_clean[df_clean['num_autores'] > 1]
        print(f"📈 Proyectos con múltiples autores: {len(proyectos_multiples)} ({len(proyectos_multiples)/len(df_clean)*100:.1f}%)")
        print(f"📊 Promedio de autores por proyecto: {df_clean['num_autores'].mean():.1f}")
        print(f"📊 Máximo de autores en un proyecto: {df_clean['num_autores'].max()}")
        
        # Análisis temporal
        if 'fecha_datetime' in df_clean.columns:
            print(f"\n📅 ANÁLISIS TEMPORAL")
            print("=" * 40)
            
            df_clean['dia_semana_num'] = df_clean['fecha_datetime'].dt.dayofweek
            proyectos_por_dia = df_clean.groupby('dia_semana').size()
            
            print(f"📊 Proyectos por día de la semana:")
            dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
            for dia, count in proyectos_por_dia.items():
                print(f"   {dia}: {count}")
            
            # Análisis por mes
            if df_clean['fecha_datetime'].dt.year.nunique() > 1:
                proyectos_por_mes = df_clean.groupby([df_clean['fecha_datetime'].dt.year, 
                                                    df_clean['fecha_datetime'].dt.month]).size()
                print(f"\n📊 Proyectos por mes:")
                for (año, mes), count in proyectos_por_mes.items():
                    print(f"   {año}-{mes:02d}: {count}")
    
    # Análisis de calidad de datos
    print(f"\n🔍 ANÁLISIS DE CALIDAD DE DATOS")
    print("=" * 40)
    
    print(f"📊 Completitud de datos:")
    for col in ['proyecto_limpio', 'fecha_limpia', 'titulo_limpio', 'partido_politico', 'tipo_proyecto']:
        if col in df_clean.columns:
            completitud = (1 - df_clean[col].isnull().sum() / len(df_clean)) * 100
            print(f"   {col}: {completitud:.1f}%")
    
    # Detectar posibles duplicados
    duplicados = df_clean.duplicated(subset=['proyecto_limpio'], keep=False)
    if duplicados.any():
        print(f"\n⚠️  Posibles duplicados detectados: {duplicados.sum()}")
        print("   Proyectos duplicados:")
        for proyecto in df_clean[duplicados]['proyecto_limpio'].unique():
            print(f"   - {proyecto}")
    else:
        print(f"\n✅ No se detectaron duplicados")
        
else:
    print("❌ No hay datos para análisis detallado")


In [ ]:
# Exportar datos limpios y resultados
if not df_clean.empty:
    
    # Guardar datos limpios
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    archivo_limpio = f'../data/proyectos_ley_limpios_{timestamp}.csv'
    df_clean.to_csv(archivo_limpio, index=False, encoding='utf-8-sig')
    print(f"💾 Datos limpios guardados en: {archivo_limpio}")
    
    # Guardar resumen en archivo de texto
    archivo_resumen = f'../analysis/resumen_analisis_{timestamp}.txt'
    with open(archivo_resumen, 'w', encoding='utf-8') as f:
        f.write("📊 RESUMEN DE ANÁLISIS - PROYECTOS DE LEY CONGRESO DEL PERÚ\n")
        f.write("=" * 70 + "\n\n")
        f.write(f"📅 Fecha de análisis: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
        f.write(f"📊 Total de proyectos: {resumen['total_proyectos']}\n")
        f.write(f"📅 Período: {resumen['fecha_inicio'].strftime('%d/%m/%Y')} - {resumen['fecha_fin'].strftime('%d/%m/%Y')}\n")
        f.write(f"👥 Promedio de autores por proyecto: {resumen['promedio_autores']:.1f}\n")
        f.write(f"🏆 Congresista más activo: {resumen['congresista_mas_activo']}\n\n")
        
        f.write("🏛️ PROYECTOS POR PARTIDO POLÍTICO:\n")
        for partido, count in resumen['proyectos_por_partido'].items():
            f.write(f"   {partido}: {count}\n")
        
        f.write(f"\n📋 PROYECTOS POR TIPO:\n")
        for tipo, count in resumen['proyectos_por_tipo'].items():
            f.write(f"   {tipo}: {count}\n")
        
        if resumen['proyectos_por_region']:
            f.write(f"\n🌍 PROYECTOS POR REGIÓN:\n")
            for region, count in resumen['proyectos_por_region'].items():
                f.write(f"   {region}: {count}\n")
    
    print(f"📄 Resumen guardado en: {archivo_resumen}")
    
    # Crear directorio de análisis si no existe
    os.makedirs('../analysis', exist_ok=True)
    os.makedirs('../visualizations', exist_ok=True)
    
    print(f"\n✅ Análisis completado exitosamente")
    print(f"📁 Archivos generados:")
    print(f"   - Datos limpios: {archivo_limpio}")
    print(f"   - Resumen: {archivo_resumen}")
    
else:
    print("❌ No hay datos para exportar")
